# Content Popularity

In this notebook we will create the Content Popularity column we will use for our dataset.

## Description:

The API has a list of keywords that have been added to the movie.

We will fetch the keywords for each title.

Then we will measure the popularity of each keyword by creating a dictionary and counting the occurence of each keyword. After we have the dictionary we will sort it and have a reference. 

Once we have the reference we can then aggregate the keyword scores and assign the value to our 'content_popularity' column.

## The Process

Here I will walk through the entire process

### Dependencies

In [214]:
# import libraries
import pandas as pd
import os
import requests
from dotenv import load_dotenv, find_dotenv
import json
import sqlite3

load_dotenv(find_dotenv())

True

API key

In [7]:
TMDB_KEY = os.getenv("TMDB_KEY")

- Title dataframe
- TMDB API (for keywords)

In [2]:
# import data dependencies
path = os.path.join(os.pardir, 'data', 'interim', 'movie_titles.csv')
titles = pd.read_csv(path)

Confirm dataframe

In [3]:
titles.sample(5)

,imdb_id,title
236995,tt0382920,Una preciosa puesta de sol
497674,tt3900072,My Story
608337,tt7051854,Marriage of Convenience
307679,tt10788800,F.C Internazionale Milano vs Manchester United
28031,tt0032778,Eternal Melodies


### Desired Shape

The resulting dataset will have three columns:
1. imdb_id (will serve as uid)
2. title
3. content_popularity

### Processing

This is going to take a few steps:
1. Gather all of the keywords associated with out dataset.
- We will use the TMDB API, so it will take some time to make all the requests.
- We will use a sqlite3 database to hold onto the values so we dont have to worry about losing the data if the computer turns off 29 hours into the program.
- Sqlite3 db will contain a row for each text:
  - imdb_id
  - keyword_name (text)
  - keyword_id
- After we have aquired all the keywords we will import the sqlite table into pandas and do the calculations for the new content_popularity value

In [168]:
def fetch_tmdb_id(imdb_id, s):
    """Calls The Movie Database API to convert IMDB id into a TMDB id
    
    Args:
        imdb_id (int): Unique identifier for IMDB
        s (Session): Session object from the Requests library
        
    Returns:
        tmdb_id (int): Unique identifier for TMDB
    """
    base_url = "https://api.themoviedb.org/3"
    payload = {'api_key': TMDB_KEY, 'external_source': 'imdb_id'}
    url = base_url + '/find/{}'.format(imdb_id)
    
    res = s.get(url, params=payload)
    res = json.loads(res.text)
    
    if len(res['movie_results']) < 1:
        return None

    try:
        if res['movie_results'][0]['id']:
            imdb_id = res['movie_results'][0]['id']
    except IndexError as e:
        print(e)

    return imdb_id
    

Let's go ahead and test it as well

In [70]:
# test fetch_tmdb_id
test_imdb_id = 'tt0137523'
fetch_tmdb_id(test_imdb_id)

550

Function to ask the API for all keywords for the specified TMDB id

In [169]:
def fetch_keywords(tmdb_id, s):
    """Calls The Movie Database API to retrieve movie keywords
    
    Args:
        tmdb_id (int): Unique identifier for TMDB (The Movie DB)
        s (Session): Session object from the Requests library

    Returns:
        keywords list(str): A list of keywords
    """
    # setup api call to grab the keywords
    base_url = "https://api.themoviedb.org/3"
    payload = {'api_key': TMDB_KEY}
    
    url = base_url + '/movie/{}/keywords'.format(tmdb_id)
    res = s.get(url, params=payload)
    res.raise_for_status()
    
    res_json = json.loads(res.text)
    
    keywords = res_json['keywords']
    
    return keywords

Let's go ahead and test it

In [386]:
# test fetch_keywords, using the TMDB it for Fight Club (550)
s = requests.Session()
fetch_keywords(550, s)

[{'id': 818, 'name': 'based on novel or book'},
 {'id': 825, 'name': 'support group'},
 {'id': 851, 'name': 'dual identity'},
 {'id': 1541, 'name': 'nihilism'},
 {'id': 1721, 'name': 'fight'},
 {'id': 3927, 'name': 'rage and hate'},
 {'id': 4142, 'name': 'insomnia'},
 {'id': 4565, 'name': 'dystopia'},
 {'id': 9181, 'name': 'alter ego'},
 {'id': 34117, 'name': 'cult film'},
 {'id': 156761, 'name': 'split personality'},
 {'id': 179173, 'name': 'quitting a job'},
 {'id': 212803, 'name': 'dissociative identity disorder'},
 {'id': 249899, 'name': 'graphic violence'},
 {'id': 260426, 'name': 'self destructiveness'}]

Function that will be used to map through the DataFrame and assign the keyword popularity value to the new column

In [34]:
def calculate_keyword_value(keywords, keyword_ratings):
    """Take list of keywords and return the average popularity
    
    Arguments:
        keywords (list): A list of keyword dicts { 'id': int, 'name': str }
        keyword_ratings (dict): A dict, mapping each keyword to its rating values
        
    Returns:
        popularity_rating (int): Aggregate (mean) score of all the keywords
    """
    
    total = 0;
    for keyword in keywords:
        total += keyword_ratings[keyword['id']]
        
    return total / len(keywords)

Using some of our functions we created, let's go ahead and create a script for building our keyword popularity dictionary

In [343]:
def create_keyword_db(imdb_ids):
    counter = 0
    for imdb_id in imdb_ids:
        # use a session object to make it faster:
        s = requests.Session()
        tmdb_id = fetch_tmdb_id(imdb_id, s)
        if tmdb_id == None:
            continue
        keywords = fetch_keywords(tmdb_id, s)
        if keywords == None:
            continue
        persist_keywords(imdb_id, keywords, 'keywords.db')
        counter += 1
        print('Added {} out of {} titles ({:%})'.format(counter, len(imdb_ids), counter / len(imdb_ids)), end='\r')
    return keywords

Pulling list of imdb_id's out of our dataframe.

We will use this list as the source of id's to query against the API

In [387]:
imdb_ids = list(titles['imdb_id'].values)
print('Our list contains {} unique imdb ids'.format(len(imdb_ids)))

Our list contains 683297 unique imdb ids


That is a lot of values to scrape. The estimated time for completion is 172 hours, which is too long.
We need a way to do it incrementally:

Let's go ahead and create a database and use a database query to find out how many imdb id's we have processed already. This will be our restarting point for when we have to interupt the program.

In [378]:
# setup the table
with sqlite3.connect('keywords.db') as con:
    cur = con.cursor()
    q = """
        CREATE TABLE keywords(
            _id INTEGER,
            imdb_id          text,
            keyword_id       int,
            keyword_name     text,
            PRIMARY KEY(_id)
        );
        """
    cur.execute(q)


In [369]:
def persist_keywords(imdb_id, keyword_list, db_name):  
    """Take our keyword object and save it to a sqlite3 db
    """
    try:
        sqliteConnection = sqlite3.connect(db_name)
        cursor = sqliteConnection.cursor()
        for keyword in keyword_list:
            cursor.execute('INSERT INTO keywords (imdb_id, keyword_id, keyword_name) VALUES (?,?,?)', (imdb_id, keyword['id'], keyword['name']))
    except sqlite3.Error as error:
        print("Error while connecting to sqlite", error)
    finally:
        if (sqliteConnection):
            sqliteConnection.commit()
            sqliteConnection.close()
            print("The SQLite connection is closed", end='\r')

To check our database table

In [388]:
# will print the contents of the db
with sqlite3.connect('keywords.db') as conn:
    cur = conn.cursor()
    q = """SELECT * FROM keywords"""
    print(cur.execute(q).fetchall())

[(1, 'tt0000009', 12193, 'reporter'), (2, 'tt0000009', 176016, 'editor'), (3, 'tt0000009', 181429, 'lost film'), (4, 'tt0000147', 154802, 'silent film'), (5, 'tt0000147', 154802, 'silent film'), (6, 'tt0000574', 154802, 'silent film'), (7, 'tt0000574', 215678, 'partially lost film'), (8, 'tt0000679', 154802, 'silent film'), (9, 'tt0000679', 154802, 'silent film'), (10, 'tt0001186', 818, 'based on novel or book'), (11, 'tt0001186', 9663, 'sequel'), (12, 'tt0001186', 12999, 'black and white'), (13, 'tt0001186', 154802, 'silent film'), (14, 'tt0001186', 181429, 'lost film'), (15, 'tt0001587', 441, 'assassination'), (16, 'tt0001630', 9673, 'love'), (17, 'tt0001630', 10614, 'tragedy'), (18, 'tt0001630', 232656, 'spreewald'), (19, 'tt0001790', 818, 'based on novel or book'), (20, 'tt0001790', 818, 'based on novel or book'), (21, 'tt0002022', 10181, 'based on play or musical'), (22, 'tt0002022', 181429, 'lost film'), (23, 'tt0002031', 497, 'william shakespeare'), (24, 'tt0002031', 14531, 'pla

In [389]:
# will print the length of the db
get_table_len('keywords.db')

26029

In [363]:
def get_table_len(db_name):
    try:
        sqliteConnection = sqlite3.connect(db_name)
        cursor = sqliteConnection.cursor()
        q = """SELECT COUNT(imdb_id) as length FROM keywords GROUP BY imdb_id;"""
        
        cursor.execute(q)
        length = cursor.fetchone()[0]
        return length
    except sqlite3.Error as error:
        print("Error while connecting to sqlite", error)
    finally:
        if (sqliteConnection):
            sqliteConnection.close()
            print("The SQLite connection is closed", end='\r')

In [391]:
import timeit
# we can use the length of items in the database as our index (stopped at 17k)
table_length = get_table_len('keywords.db')
%timeit create_keyword_db(imdb_ids[table_length:])


KeyboardInterrupt: 

We need to persist this somehow. It is too long of a process to be able to reliably store it in memory.

Now that we have a list of keywords and how often they appear, we can decide how we will determine the popularity value.

It makes sense to make it a ratio of occurences to total number of words.

<code>keyword = { 'strong female lead': 25 }
total_keywords = 100
popularity_value = keyword['strong female lead'] / total_keywords)
print(popularity_value)
</code>
Will print out .25